Sampler

In [1]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from sage.stats.distributions.discrete_gaussian_lattice import DiscreteGaussianDistributionLatticeSampler
import sage.matrix.matrix_integer_dense_hnf as HNF
    
def Norm_Q(Q, u):
    return sqrt(InnerProduct_Q(Q, u, u))

def Norm_eucl_2(v):
    return v.dot_product(v)

def InnerProduct_Q(Q, u, v):
    return u * Q * v

def Proj(Q, u, v):
    #Projects v onto u using Q-inner product (Q-norm)
    return (InnerProduct_Q(Q, u, v) / InnerProduct_Q(Q, u, u)) * u

def GramSchmidt_Q(Q, B):
    N = Q.dimensions()[0]
    B_gram = [zero_vector(N) for _ in range(N)]
    for i in range(N):
        B_gram[i] = B[i]
        for j in range(i):
            B_gram[i] = B_gram[i] - Proj(Q, B_gram[j], B_gram[i])
    
    return B_gram

def GramSchmidt(B):
    #Sagemath GSO (for Q-forms sampling)
    return B.gram_schmidt(orthonormal=False)[0]

def SortByColumnNorm(B):
    N = B.dimensions()[0]
    columns = B.columns()
    columns.sort(key=lambda v : Norm_eucl_2(v))
    return block_matrix([[matrix(column).transpose() for column in columns]])

def Rho(x, s, c):
    return exp(-pi * Norm_eucl_2(x - c)/ s^2)


def InsertColumn(A, c, v):
    N, M = A.dimensions()
    for i in range(N):
        A[i,c] = v[i]


'''
Sage samples with exp(-x**2/(2*s**2)).
Need exp(-pi*||x||**2/s**2) -> rescale the sampling width by 1/sqrt(2*pi).
'''
def SampleZ(s, c):
    D = DiscreteGaussianDistributionIntegerSampler(sigma=s/sqrt(2*pi), c=c)
    return D()


#Q-norm Gaussian sampler
def SampleD_c(Q, B, B_, s, c):
    N = Q.dimensions()[0]
    
    vi = zero_vector(ZZ, N)
    ci = vector(c)
    
    for i in range(N-1, -1, -1):
        bi_ = B_[i]
        c_ = InnerProduct_Q(Q, ci, bi_) / InnerProduct_Q(Q, bi_, bi_)
        s_ = s / Norm_Q(Q, bi_)
        
        zi = SampleZ(s_, c_)
        bi = B[i]
        ci = ci - zi * bi
        vi = vi + zi * bi

    return vi


#Q-forms sampling
def FirstN(V):
    """
    First N linearly independent of vectors |v1| <= |v2| <= ... <= |vN| (SortByColumnNorm).
    """
    N, M = V.dimensions()
    V_ = matrix(QQ, N, 0)

    for column in V.columns():
        aux = block_matrix([[V_, matrix(column).transpose()]])

        if aux.rank() > V_.rank():
            V_ = aux

        if V_.rank() == N:
            return V_
    
    
def Extract(B, V):
    N, M = V.dimensions()

    Y = matrix(ZZ, B.inverse() * V)
    H, U_ = HNF.hnf_with_transformation(Y) #H = Y*U^(-1)
    
    print('HNF DONE')

    U = matrix(ZZ, U_.inverse())
    R = B * U
    
    print('INV+MULT DONE')
    #assert V == R * H
    V_ = GramSchmidt(V)
    
    print('GramSchmidt DONE -> slow')

    for k in range(N):
        v = V.column(k)
        r = R.column(k)

        if Norm_eucl_2(r) <= max((k+1)/4, 1) * Norm_eucl_2(v):
            continue

        R_ = GramSchmidt(R)

        r_ = R_.column(k)
        v_ = V_.column(k)

        if r_ == v_ or r_ == -v_:
            InsertColumn(R, k, v)
        else:
            x = r - r_
            BR = block_matrix([[matrix(R.column(i)).transpose() for i in range(k)]])
            v_pr = NearestPlane(BR, x)
            InsertColumn(R, k, r - v_pr)

    # R is modified -> recompute U such that R = B * U
    U = matrix(ZZ, N, N, B.inverse() * R)
    return (R.transpose() * R, U)



def SampleQuadraticForm(B, s):
    """
    Alg 8 from our file.
    """
    print("Sampling U")
    N = B.dimensions()[0]
    C = 1 - (1 + exp(-pi))^(-1)
    m = ceil(2*N / C)
    c = zero_vector(ZZ, N)
    
    #Decrease m while testing
    #m = m//2

    D = DiscreteGaussianDistributionLatticeSampler(B.transpose(), s/sqrt(2*3.14), zero_vector(N))
    V = matrix(ZZ, m, N, [D() for _ in range(m)]).transpose()

    if V.rank() < N:
        return SampleQuadraticForm(B, s)
    
    print('SAMPLING DONE -> slow')
    V = FirstN(V)
    print('SIMPLIFY DONE')
    V = SortByColumnNorm(V)
    print('SORT DONE')
    R, U = Extract(B, V)
    return (R, U, V)
    


#BW construction
def BW_n(n):
    phi = 1 + i 
    A = matrix([[1, 0], [1, phi]])
    #print(A)
    result = A
    for _ in range(n - 1):
        result = result.tensor_product(A)   
    return result


def complex_to_2x2_block(a_plus_ib):
    a = a_plus_ib.real()
    b = a_plus_ib.imag()
    return matrix([[a, b], [-b, a]])

def matrix_integer(M):
    n = M.nrows()
    new_size = 2 * n
    new_matrix = matrix(QQ, new_size, new_size, 0)
    
    for i in range(n):
        for j in range(n):
            block = complex_to_2x2_block(M[i, j])
            for bi in range(2):
                for bj in range(2):
                    new_matrix[2*i + bi, 2*j + bj] = block[bi, bj]
    
    return new_matrix

Q-norm 

In [2]:
k = 5
n = 2^(k+1)

BW_complex = BW_n(k)

BW = matrix_integer(BW_complex)

Q = BW.T*BW

B = identity_matrix(n) #Z^n

B_gram = GramSchmidt_Q(Q, B)# > n^4 multiplications (compute -> hard code?)

print(matrix(B_gram).dimensions())

#print(list(B))

(64, 64)


In [3]:
s = 3

q = 9847

t = vector([randint(1,q) for _ in range(n)])/q
t = t + vector([randint(1,q) for _ in range(n)])

sig = SampleD_c(Q, B, B_gram, s, t)  

print(sig)

print(Norm_Q(Q,t-sig).n())

print((BW*(t-sig)).norm().n())

print((s*sqrt(n)).n())

(2670, 3993, 5817, 7649, 7432, 3067, 4642, 6530, 5733, 61, 7593, 3588, 6072, 8212, 7516, 8493, 5763, 6439, 6686, 1302, 632, 65, 6370, 7191, 3931, 5727, 4332, 2219, 2782, 774, 1347, 9108, 3889, 866, 8417, 7742, 5074, 9336, 3244, 4327, 207, 3055, 1227, 9832, 1393, 4028, 3286, 3785, 67, 5556, 8519, 4410, 5989, 6674, 555, 1917, 380, 6035, 3274, 9258, 5158, 9386, 7422, 2098)
9.42515299934885
9.42515299934885
24.0000000000000


Euclidean norm

In [4]:
#BW construction
k = 5
n = 2^(k+1)
BW = matrix_integer(BW_n(k))

#Quadratic forms sampling
P, U, S = SampleQuadraticForm(BW, 30)

Sampling U
SAMPLING DONE -> slow
SIMPLIFY DONE
SORT DONE
HNF DONE
INV+MULT DONE
GramSchmidt DONE -> slow


In [5]:
q = 9847

#Target 
t = vector([randint(1,(q-1)) for _ in range(n)])/q + vector([randint(1,(q-1)) for _ in range(n)])

#Gaussian distribution params
s = 3
cent = BW*U*t #new center

D = DiscreteGaussianDistributionLatticeSampler(BW, sigma = s/sqrt(2*3.14), c = cent)

sig_ = D()


sig = U.inverse()*BW.inverse()*sig_

#print("sig = ", sig , end='\n\n')

print(Norm_Q(P, sig-t).n(),'Sig P-norm') #Condition to check in ver function
print((t-sig).norm().n(),'Sig eucl norm')

#Check
print((BW*U*sig-BW*U*t).norm().n(), 'P-norm of signature') #P-norm of signature
print((sig_-cent).norm().n(), 'S-norm of signature',end="\n\n") #S-norm of signature, cent \in sk


print("Restriction on norm")
print((s*sqrt(n).n()), end="\n\n") #Restriction on norm


print((BW*U*(sig-t)).norm().n())# now (sig-t) \in Q^n (need Z^n?), norm of BW'*(sig-t) is small (BW' = BW*U) 
#Signature forgery <-> find short vector in BW*U with center t <-> hard as long as U \in sk (BW*U is unknown)

#print((BW*(sig_-cent)).norm().n())

9.10258994340492 Sig P-norm
1.41947918053786e86 Sig eucl norm
9.10258994340492 P-norm of signature
9.10258994340492 S-norm of signature

Restriction on norm
24.0000000000000

9.10258994340492
